In [1]:
import numpy as np
import tensorflow as tf
tf.GraphKeys.VARIABLES = tf.GraphKeys.GLOBAL_VARIABLES
import pandas as pd
import csv
import math
import shutil
import os
import time

In [2]:
def inception1(input_tensor):
    ## 2*2
    conv2_2 = tf.layers.conv2d( input_tensor,filters = 128, kernel_size = 2, strides = 1,padding = "same",
                                    activation = tf.nn.relu, use_bias = True, 
                                    kernel_initializer = tf.contrib.layers.xavier_initializer_conv2d(),
                                    bias_initializer = tf.contrib.layers.xavier_initializer_conv2d())
    conv2 = tf.layers.conv2d( conv2_2,filters = 128, kernel_size = 2, strides = 1,padding = "same",
                                    activation = tf.nn.relu, use_bias = True, 
                                    kernel_initializer = tf.contrib.layers.xavier_initializer_conv2d(),
                                    bias_initializer = tf.contrib.layers.xavier_initializer_conv2d())
    ## 3*3
    conv3_3 = tf.layers.conv2d( input_tensor,filters = 128, kernel_size = 3, strides = 1,padding = "same",
                                    activation = tf.nn.relu, use_bias = True, 
                                    kernel_initializer = tf.contrib.layers.xavier_initializer_conv2d(),
                                    bias_initializer = tf.contrib.layers.xavier_initializer_conv2d())
    conv3 = tf.layers.conv2d( conv3_3,filters = 128, kernel_size = 3, strides = 1,padding = "same",
                                    activation = tf.nn.relu, use_bias = True, 
                                    kernel_initializer = tf.contrib.layers.xavier_initializer_conv2d(),
                                    bias_initializer = tf.contrib.layers.xavier_initializer_conv2d())
    ##5*5
    conv5_5 = tf.layers.conv2d( input_tensor,filters = 128, kernel_size = 5, strides = 1,padding = "same",
                                activation = tf.nn.relu, use_bias = True, 
                                kernel_initializer = tf.contrib.layers.xavier_initializer_conv2d(),
                                bias_initializer = tf.contrib.layers.xavier_initializer_conv2d())
    conv5 = tf.layers.conv2d( conv5_5,filters = 128, kernel_size = 5, strides = 1,padding = "same",
                                activation = tf.nn.relu, use_bias = True, 
                                kernel_initializer = tf.contrib.layers.xavier_initializer_conv2d(),
                                bias_initializer = tf.contrib.layers.xavier_initializer_conv2d())
    
    combine_tensor1 = tf.concat([input_tensor,conv2],2)
    combine_tensor2 = tf.concat([conv3, conv5],2)
    output_tensor = tf.concat([combine_tensor1, combine_tensor2],1)
    
    return output_tensor

In [3]:
tf.reset_default_graph()

In [4]:
x = tf.placeholder(tf.float32,shape = [None,13*13*9])
y_ = tf.placeholder(tf.int64,shape = [None])
y_one_hot = tf.one_hot(y_, 2)
x_image = tf.reshape(x,[-1,13,13,9])
train_key = tf.placeholder(bool)
with tf.variable_scope("PHM"):
    ## siganl preprocess
    result_conv1 = tf.layers.conv2d( x_image,filters = 16, kernel_size = 1, strides = 1,padding = "same",
                                    activation = tf.nn.relu, use_bias = True,
                                    kernel_initializer = tf.contrib.layers.xavier_initializer_conv2d(),
                                    bias_initializer = tf.contrib.layers.xavier_initializer_conv2d())
    result_conv2 = tf.layers.conv2d( result_conv1,filters = 32, kernel_size = 1, strides = 1,padding = "same",
                                    activation = tf.nn.relu, use_bias = True, 
                                    kernel_initializer = tf.contrib.layers.xavier_initializer_conv2d(),
                                    bias_initializer = tf.contrib.layers.xavier_initializer_conv2d())
    
    result_conv3 = tf.layers.conv2d( result_conv2,filters = 64, kernel_size = 1, strides = 1,padding = "same",
                                    activation = tf.nn.relu, use_bias = True, 
                                    kernel_initializer = tf.contrib.layers.xavier_initializer_conv2d(),
                                    bias_initializer = tf.contrib.layers.xavier_initializer_conv2d())

    result_conv4 = tf.layers.conv2d( result_conv3,filters = 128, kernel_size = 1, strides = 1,padding = "same",
                                    activation = tf.nn.relu, use_bias = True, 
                                    kernel_initializer = tf.contrib.layers.xavier_initializer_conv2d(),
                                    bias_initializer = tf.contrib.layers.xavier_initializer_conv2d())
    
    
    
    
    result_conv5 = inception1(result_conv4)

    result_conv5_1 =  tf.layers.conv2d( result_conv5,filters = 176, kernel_size = 1, strides = 1,padding = "same",
                                    activation = tf.nn.relu, use_bias = True, 
                                    kernel_initializer = tf.contrib.layers.xavier_initializer_conv2d(),
                                    bias_initializer = tf.contrib.layers.xavier_initializer_conv2d())
 
    result_conv6 = tf.layers.conv2d(result_conv5_1,filters = 194, kernel_size = 13, strides = 13,padding = "valid",
                                    activation = tf.nn.relu, use_bias = True, 
                                    kernel_initializer = tf.contrib.layers.xavier_initializer_conv2d(),
                                    bias_initializer = tf.contrib.layers.xavier_initializer_conv2d())


   
    result_conv6_1 = tf.layers.conv2d( result_conv6,filters = 256, kernel_size = 1, strides = 1,padding = "same",
                                    activation = tf.nn.relu, use_bias = True, 
                                    kernel_initializer = tf.contrib.layers.xavier_initializer_conv2d(),
                                    bias_initializer = tf.contrib.layers.xavier_initializer_conv2d())
    
    result_conv6_1_1 = tf.layers.conv2d( result_conv6_1,filters = 512, kernel_size = 1, strides = 1,padding = "same",
                                    activation = tf.nn.relu, use_bias = True, 
                                    kernel_initializer = tf.contrib.layers.xavier_initializer_conv2d(),
                                    bias_initializer = tf.contrib.layers.xavier_initializer_conv2d())    
    
    
    result_conv6_flat = tf.reshape( result_conv6_1_1 , [-1, 2 * 2 * 512])
    
    
    result_dense6 = tf.layers.dense(inputs= result_conv6_flat, units = 1024, activation=tf.nn.relu, use_bias=True,
                                    kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                    bias_initializer=tf.contrib.layers.xavier_initializer())
    result_dense6_drop = tf.layers.dropout(result_dense6, rate=0.5, noise_shape=None, seed=None, training=train_key, name=None)
    
    # Logits Layer
    before_output = tf.layers.dense(inputs = result_dense6_drop, units=2, activation = None, use_bias=True,
                                    kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                    bias_initializer=tf.contrib.layers.xavier_initializer())
    flat_output = tf.nn.softmax(before_output)
    

##L2 regularization

trainable_var_key = tf.GraphKeys.TRAINABLE_VARIABLES
all_vars = tf.get_collection(key=trainable_var_key, scope="PHM")
l2_losses = [tf.nn.l2_loss(w) for w in all_vars]
sum_l2_losses =tf.add_n(l2_losses)
    
##corss_entropy

cross_entropy = tf.reduce_mean(tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(logits=before_output , labels=y_one_hot))) + 0.1*sum_l2_losses
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(before_output,1), tf.argmax(y_one_hot,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [5]:
def read_indexfile(file_path):
    record_fault = np.genfromtxt((file_path+"/start_fault_list.csv"), delimiter=',', dtype=np.int32)
    record_file =  np.genfromtxt((file_path+"/start_plant_list.csv"), delimiter=',', dtype=np.int32)
    return record_fault,record_file


In [6]:
def create_train_validate(train_ori):
    train_ori_1 = train_ori.loc[train_ori.start_label == 1,:]
    train_ori_1.index = range(train_ori_1.shape[0])
    train_ori_0 = train_ori.loc[train_ori.start_label == 0,:]
    train_ori_0.index = range(train_ori_0.shape[0])

    trainset_1_pick = np.random.choice(train_ori_1.shape[0], math.ceil(train_ori_1.shape[0]/8), replace=False)
    trainset_1 = train_ori_1.drop(trainset_1_pick)
    validate_1 = train_ori_1.loc[trainset_1_pick,:]

    trainset_0_pick = np.random.choice(train_ori_0.shape[0], math.ceil(train_ori_0.shape[0]/8), replace=False)
    trainset_0 = train_ori_0.drop(trainset_0_pick)
    validate_0 = train_ori_0.loc[ trainset_0_pick,: ]
    print("trainset_1: %s trainset_0:%s" % (trainset_1.shape[0],trainset_0.shape[0]))
    ##do down sample
    if(trainset_0.shape[0]>trainset_1.shape[0]):
        trainset_0.index = range(trainset_0.shape[0])
        sample_pick = np.random.choice(trainset_0.shape[0],math.ceil(trainset_1.shape[0]*1.4),replace = False)
        trainset_0 = trainset_0.loc[sample_pick,:]
    trainset = pd.concat([trainset_1,trainset_0], axis = 0, ignore_index = True)
    validate = pd.concat([validate_1,validate_0], axis = 0, ignore_index = True)
    print("trainset: %s validate: %s"%(trainset.shape[0], validate.shape[0]))
    return trainset,validate

In [7]:
def best_model(acc_validate,dir_path):
    best_acc = max(acc_validate)
    for index in range( len(acc_validate) ):
        if acc_validate[index] == best_acc:
            break
    best_name = dir_path+"/model" + str(index)+".ckpt"
    print(best_name)
    return best_name

In [8]:
def write_record(filename,record):
    f = open(filename,"w")
    f.write( str(record[0]) )
    for index in range(1,len(record)):
        f.write( ","+str(record[index]))
    f.close()

In [ ]:
def pred_run(read_file_path,write_file_path,epoch_times,train_batch_size,test_batch_size):
## read file
    record_fault, record_file = read_indexfile(read_file_path)
    if not os.path.exists(write_file_path):
        os.makedirs(write_file_path )
    
    if not os.path.exists(write_file_path + "/pred_start"):
        os.makedirs(write_file_path + "/pred_start")
    
    if not os.path.exists( (write_file_path + "/pred_start/validate") ):
        os.makedirs( (write_file_path + "/pred_start/validate") ) 
## loop run
    ##launch model
    ##record_usefault_file
    vali_record = []
    for file_index in range(0, 2):
        
        print("index: %s , total:%s, plant:%s, fault:%s" % (file_index,len(record_fault),record_file[file_index], record_fault[file_index]))
        train_filename = read_file_path+"/start/trainset"+str(record_file[file_index])+"_fault"+str(record_fault[file_index])+".csv"
        test_filename = read_file_path+"/start/testset"+str(record_file[file_index])+"_fault"+str(record_fault[file_index])+".csv"

        train_ori = pd.read_csv(train_filename)
        test_ori = pd.read_csv(test_filename)
        train_ori_one = train_ori.loc[train_ori["start_label"]==1,:]

        if train_ori_one.shape[0] > 199:

            ## vali_record
            vali_record.append(1)
            vali_record_name = write_file_path+"/pred_start/record_vali.csv"
            write_record(vali_record_name,vali_record)

            

            trainset_raw, validate_raw = create_train_validate(train_ori)

            print(trainset_raw.shape)
            trainset = trainset_raw.drop(["time","start_label"],axis = 1)
            validate = validate_raw.drop(["time","start_label"],axis = 1)
            validate_record = validate_raw
            testset = test_ori.drop(["time","start_label"],axis = 1)

            trainset_one_ori = trainset_raw.loc[trainset_raw.start_label == 1, :]
            trainset_one_ori.index = range(trainset_one_ori.shape[0])
            trainset_one = trainset_one_ori.drop(["time", "start_label"], axis = 1)

            testset_one_ori = test_ori.loc[test_ori.start_label == 1,:]
            testset_one_ori.index = range(testset_one_ori.shape[0])
            testset_one = testset_one_ori.drop(["time","start_label"],axis = 1)

            validate_one_ori = validate_raw.loc[validate_raw.start_label == 1,:]
            validate_one_ori.index = range(validate_one_ori.shape[0])
            validate_one = validate_one_ori.drop(["time","start_label"],axis = 1)

            best_vali_acc = 0
            best_score = 0
            best_gap = 0
            best_zero_one = 0
            best_zero_all = 0
            best_one_all = 0
            ##early stop record
            record_train = []
            record_vali = []
            record_vali_one = []
            record_train_one = []
            avg_train = 0
            avg_train_one = 0
            avg_vali = 0
            avg_vali_one =0
            
            ##initial model
            config = tf.ConfigProto()
            config.gpu_options.allocator_type = 'BFC'
            with tf.Session(config = config) as sess:
            #sess = tf.InteractiveSession()
                sess.run(tf.global_variables_initializer())
                ##train
                ##model saver path
                dir_name = "/plant"+str(record_file[file_index])+"fault"+str(record_fault[file_index])
                dir_path = write_file_path + "/start_model_saver"+dir_name
                if os.path.exists(dir_path):
                    shutil.rmtree(dir_path)
                os.makedirs(dir_path)
                saver = tf.train.Saver()
                
                ##start training
                for epoch in range(epoch_times):
                    ##training
                    index_r = np.random.choice(trainset.shape[0],trainset.shape[0],replace = False)
                    for index in range( math.ceil(trainset.shape[0]/train_batch_size)):
                        if index < (math.ceil(trainset.shape[0]/train_batch_size)-1):
                            batch_index = index_r[ [ i for i in range(index*train_batch_size,(index+1)*train_batch_size) ] ]
                            input_train = trainset.iloc[ batch_index,: ]
                            input_label = trainset_raw.start_label[batch_index]
                            train_step.run(feed_dict={x: input_train.values, y_: input_label.values,train_key: True})
                        else:
                            batch_index = index_r[ [ i for i in range(index*train_batch_size, len(index_r) )] ]
                            input_train = trainset.iloc[ batch_index,: ]
                            input_label = trainset_raw.start_label[batch_index]
                            train_step.run(feed_dict={x: input_train.values, y_: input_label.values,
                                                      train_key: True})
                            
                    ##test testing set
                    acc_test_all = 0
                    for index in range(math.ceil(testset.shape[0]/train_batch_size)):
                        if index < (math.ceil(testset.shape[0]/train_batch_size)-1):
                            batch_index = [ i for i in range(index*train_batch_size,(index+1)*train_batch_size) ]
                            input_train = testset.iloc[ batch_index,: ]
                            input_label = test_ori.start_label[batch_index]
                            acc_vali_part = sess.run([accuracy],
                                                      feed_dict={x: input_train.values, y_: input_label.values,train_key: False})                                     
                            acc_test_all = acc_test_all + acc_vali_part[0]*train_batch_size
                        else:
                            batch_index = [ i for i in range(index*train_batch_size,testset.shape[0]) ]
                            input_train = testset.iloc[ batch_index,: ]
                            input_label = test_ori.start_label[batch_index]
                            acc_vali_part = sess.run([accuracy],
                                                    feed_dict={x: input_train.values, y_: input_label.values,train_key: False})                            
                            acc_test_all = acc_test_all + acc_vali_part[0]*len(batch_index)

                    acc_test_all = acc_test_all / testset.shape[0]

                    ##test testing set in label one
                    acc_test_one_all = 0
                    if(testset_one.shape[0]>0):
                        for index in range(math.ceil(testset_one.shape[0]/train_batch_size)):
                            if index < (math.ceil(testset_one.shape[0]/train_batch_size)-1):
                                batch_index = [ i for i in range(index*train_batch_size,(index+1)*train_batch_size) ]
                                input_train = testset_one.iloc[ batch_index,: ]
                                input_label = testset_one_ori.start_label[batch_index]
                                acc_vali_part = sess.run([accuracy],
                                                          feed_dict={x: input_train.values, y_: input_label.values,train_key: False})                                     
                                acc_test_one_all = acc_test_one_all + acc_vali_part[0]*train_batch_size
                            else:
                                batch_index = [ i for i in range(index*train_batch_size,testset_one.shape[0]) ]
                                input_train = testset_one.iloc[ batch_index,: ]
                                input_label = testset_one_ori.start_label[batch_index]
                                acc_vali_part = sess.run([accuracy],
                                                        feed_dict={x: input_train.values, y_: input_label.values,train_key: False})                            
                                acc_test_one_all = acc_test_one_all + acc_vali_part[0]*len(batch_index)

                        acc_test_one_all = acc_test_one_all / testset_one.shape[0]
                    
                    #test validating set
                    acc_vali_all = 0
                    for index in range(math.ceil(validate.shape[0]/train_batch_size)):
                        if index < (math.ceil(validate.shape[0]/train_batch_size)-1):
                            batch_index = [ i for i in range(index*train_batch_size,(index+1)*train_batch_size) ]
                            input_train = validate.iloc[ batch_index,: ]
                            input_label = validate_raw.start_label[batch_index]
                            acc_vali_part = sess.run([accuracy],
                                                      feed_dict={x: input_train.values, y_: input_label.values,train_key: False})                                     
                            acc_vali_all = acc_vali_all + acc_vali_part[0]*train_batch_size
                        else:
                            batch_index = [ i for i in range(index*train_batch_size,validate.shape[0]) ]
                            input_train = validate.iloc[ batch_index,: ]
                            input_label = validate_raw.start_label[batch_index]
                            acc_vali_part = sess.run([accuracy],
                                                    feed_dict={x: input_train.values, y_: input_label.values,train_key: False})                            
                            acc_vali_all = acc_vali_all + acc_vali_part[0]*len(batch_index)

                    acc_vali_all = acc_vali_all / validate.shape[0]
                    record_vali.append(acc_vali_all)
                    
                    #test validating set in label one
                    acc_vali_one_all = 0
                    if (validate_one.shape[0]>0):
                        for index in range(math.ceil(validate_one.shape[0]/train_batch_size)):
                            if index < (math.ceil(validate_one.shape[0]/train_batch_size)-1):
                                batch_index = [ i for i in range(index*train_batch_size,(index+1)*train_batch_size) ]
                                input_train = validate_one.iloc[ batch_index,: ]
                                input_label = validate_one_ori.start_label[batch_index]
                                acc_vali_part = sess.run([accuracy],
                                                          feed_dict={x: input_train.values, y_: input_label.values,train_key: False})                                     
                                acc_vali_one_all = acc_vali_one_all + acc_vali_part[0]*train_batch_size
                            else:
                                batch_index = [ i for i in range(index*train_batch_size,validate_one.shape[0]) ]
                                input_train = validate_one.iloc[ batch_index,: ]
                                input_label = validate_one_ori.start_label[batch_index]
                                acc_vali_part = sess.run([accuracy],
                                                        feed_dict={x: input_train.values, y_: input_label.values,train_key: False})                            
                                acc_vali_one_all = acc_vali_one_all + acc_vali_part[0]*len(batch_index)

                        acc_vali_one_all = acc_vali_one_all / validate_one.shape[0]
                    record_vali_one.append(acc_vali_one_all)
                    
                    ##test training set
                    acc_train_all = 0
                    for index in range(math.ceil(trainset.shape[0]/train_batch_size)):
                        if index < (math.ceil(trainset.shape[0]/train_batch_size)-1):
                            batch_index = [ i for i in range(index*train_batch_size,(index+1)*train_batch_size) ]
                            input_train = trainset.iloc[ batch_index,: ]
                            input_label = trainset_raw.start_label[batch_index]
                            acc_train_part = sess.run([accuracy],
                                                      feed_dict={x: input_train.values, y_: input_label.values,train_key: False})                                     
                            acc_train_all = acc_train_all + acc_train_part[0]*train_batch_size
                        else:
                            batch_index = [ i for i in range(index*train_batch_size,trainset.shape[0]) ]
                            input_train = trainset.iloc[ batch_index,: ]
                            input_label = trainset_raw.start_label[batch_index]
                            acc_train_part = sess.run([accuracy],
                                                    feed_dict={x: input_train.values, y_: input_label.values,train_key: False})                            
                            acc_train_all = acc_train_all + acc_train_part[0]*len(batch_index)
                    acc_train_all = acc_train_all / trainset.shape[0]
                    record_train.append(acc_train_all)

                    #test training set in label one
                    acc_train_one_all = 0
                    for index in range(math.ceil(trainset_one.shape[0]/train_batch_size)):
                        if index < (math.ceil(trainset_one.shape[0]/train_batch_size)-1):
                            batch_index = [ i for i in range(index*train_batch_size,(index+1)*train_batch_size) ]
                            input_train = trainset_one.iloc[ batch_index,: ]
                            input_label = trainset_one_ori.start_label[batch_index]
                            acc_train_part = sess.run([accuracy],
                                                      feed_dict={x: input_train.values, y_: input_label.values,train_key: False})                                     
                            acc_train_one_all = acc_train_one_all + acc_train_part[0]*train_batch_size
                        else:
                            batch_index = [ i for i in range(index*train_batch_size,trainset_one.shape[0]) ]
                            input_train = trainset_one.iloc[ batch_index,: ]
                            input_label = trainset_one_ori.start_label[batch_index]
                            acc_train_part = sess.run([accuracy],
                                                    feed_dict={x: input_train.values, y_: input_label.values,train_key: False})                            
                            acc_train_one_all = acc_train_one_all + acc_train_part[0]*len(batch_index)
                            
                    ##calculate the acc in label zero
                    acc_train_one_all = acc_train_one_all / trainset_one.shape[0]
                    record_train_one.append(acc_train_one_all)
                    acc_vali_zero_all = (acc_vali_all*validate.shape[0] - acc_vali_one_all*validate_one.shape[0])/(validate.shape[0]-validate_one.shape[0])
                    zero_one = acc_vali_zero_all * acc_vali_one_all

                    acc_test_zero_all = (acc_test_all*testset.shape[0] - acc_test_one_all *testset_one.shape[0]) / (testset.shape[0] - testset_one.shape[0])
                    now_score = validate_one.shape[0]*10*acc_vali_one_all - (validate.shape[0]-validate_one.shape[0])*(1-acc_vali_all)*0.1 
                    print("epoch %d, vali: %.3g, vali_o: %.3g zero: %.3g *: %.3g train: %.3g test: %.3g test_o: %.3g zero: %.3g"%(epoch, acc_vali_all, acc_vali_one_all,acc_vali_zero_all, zero_one, acc_train_all, acc_test_all, acc_test_one_all, acc_test_zero_all))


                    ##record the model
                    if (epoch > 3 ):
                        if(epoch > 5):
                            ##early stop
                            avg_train_now = sum(record_train[(epoch-4):(epoch+1)])/5
                            avg_vali_now = sum(record_vali[(epoch-4):(epoch+1)])/5
                            avg_vali_one_now = sum(record_vali_one[(epoch-5):(epoch+1)])

                        #if (now_score > best_score and acc_vali_all > best_vali_acc):
                        #if( (acc_vali_all > best_vali_acc and zero_one > best_zero_one) or (acc_vali_all  >0.85 and acc_vali_all> acc_train_all)):
                        ##if( acc_vali_all > best_vali_acc and zero_one > best_zero_one and acc_vali_zero_all >best_zero_all ):
                        ##if(  zero_one > best_zero_one and acc_vali_zero_all >best_zero_all ):
                        if(  zero_one > best_zero_one and acc_vali_all >best_vali_acc ):
                            best_zero_one = zero_one
                            best_vali_acc = acc_vali_all
                            best_zero_all = acc_vali_zero_all
                            best_one_all = acc_vali_one_all
                            if(epoch > 5):
                                avg_trian = sum(record_train[(epoch-4):(epoch+1)])/5
                                avg_vali = sum(record_vali[(epoch-4):(epoch+1)])/5
                                avg_vali_one = sum(record_vali_one[(epoch-5):(epoch+1)])
                            print("best one !!!!")
                            best_score = now_score
                            if os.path.exists(dir_path):
                                shutil.rmtree(dir_path)
                            os.makedirs(dir_path)
                            model_name = dir_path+"/model.ckpt"
                            saver.save(sess, model_name)

                        if(epoch > 5):
                            """
                            if( (avg_vali_one_now/avg_vali_one)<0.97):
                                print("early stop vali one")
                                break

                            if((avg_vali_now/avg_vali) < 0.97):
                                print("early stop vali")
                                break
                            
                            if(avg_train_now>0.85):
                                if((avg_vali_now/avg_train_now)<0.96):
                                    print("early stop train > vali so far")
                                    break
                            """
                            if(avg_train_now > 0.99):
                                print("overfitting")
                                break
                            

                ##test
                ##load the best model
                saver.restore(sess,  model_name )
                ##predict validating set
                pred = []
                for i in range( math.ceil(validate.shape[0]/test_batch_size )):
                    if((i+1)*train_batch_size < validate.shape[0]): 
                        temp_pred = sess.run([flat_output], feed_dict={
                            x: validate.iloc[ (i*train_batch_size) : ((i+1)*train_batch_size),: ].values,train_key: False})
                    else:
                        ##pdb.set_trace()
                        temp_pred = sess.run([flat_output], feed_dict={
                            x: validate.iloc[ (i*train_batch_size) : testset.shape[0] ].values,train_key: False})
                    pred.extend(temp_pred[0])
                zero_list = []
                one_list = []
                for array in pred:
                    zero_list.append(array[0])
                    one_list.append(array[1])

                df = {"zero":zero_list, "one":one_list, "true": validate_raw.start_label, "time":validate_raw.time}
                pred_data = pd.DataFrame(df)
                pred_data_name = write_file_path + "/pred_start/validate/start_vali_5conv"+str(record_file[file_index])+"_fault"+str(record_fault[file_index])+".csv"
                pred_data.to_csv(pred_data_name )

                ##predict testing set  
                pred = []
                for i in range( math.ceil(testset.shape[0]/test_batch_size )):
                    if((i+1)*train_batch_size < testset.shape[0]): 
                        temp_pred = sess.run([flat_output], feed_dict={
                            x: testset.iloc[ (i*train_batch_size) : ((i+1)*train_batch_size),: ].values,train_key: False})
                    else:
                        temp_pred = sess.run([flat_output], feed_dict={
                            x: testset.iloc[ (i*train_batch_size) : testset.shape[0] ].values,train_key: False})
                    pred.extend(temp_pred[0])

                zero_list = []
                one_list = []
                for array in pred:
                    zero_list.append(array[0])
                    one_list.append(array[1])
                df = {"zero":zero_list, "one":one_list, "true": test_ori.start_label, "time": test_ori.time}
                pred_data = pd.DataFrame(df)
                pred_data_name = write_file_path + "/pred_start/start_pred_5conv"+str(record_file[file_index])+"_fault"+str(record_fault[file_index])+".csv"
                pred_data.to_csv(pred_data_name,index = False,sep = "," )
            sess.close()

        else:
            vali_record.append(0)
            vali_record_name = write_file_path+"/pred_start/record_vali.csv"
            write_record(vali_record_name,vali_record)
    vali_record_name = write_file_path+"/pred_start/record_vali.csv"
    write_record(vali_record_name,vali_record)

            
            

In [ ]:
read_file_path = "/media/joeytu/82F6C88FF6C8853F/PHM2017/PHM_nosample/file"
write_file_path = "/media/joeytu/82F6C88FF6C8853F/PHM2017/20170831DEMO/file"
epoch_times = 5
train_batch_size = 300
test_batch_size = 300
pred_run(read_file_path ,write_file_path,epoch_times, train_batch_size, test_batch_size)

index: 0 , total:135, plant:1, fault:1
trainset_1: 4760 trainset_0:63644
trainset: 11424 validate: 9774
(11424, 1523)
epoch 0, vali: 0.718, vali_o: 0.852 zero: 0.708 *: 0.603 train: 0.775 test: 0.591 test_o: 0.949 zero: 0.569
epoch 1, vali: 0.767, vali_o: 0.91 zero: 0.756 *: 0.688 train: 0.833 test: 0.564 test_o: 0.979 zero: 0.538
epoch 2, vali: 0.795, vali_o: 0.927 zero: 0.786 *: 0.728 train: 0.852 test: 0.556 test_o: 0.984 zero: 0.53
epoch 3, vali: 0.897, vali_o: 0.698 zero: 0.912 *: 0.636 train: 0.838 test: 0.672 test_o: 0.892 zero: 0.658
epoch 4, vali: 0.855, vali_o: 0.891 zero: 0.852 *: 0.759 train: 0.879 test: 0.591 test_o: 0.977 zero: 0.567
best one !!!!
index: 1 , total:135, plant:1, fault:2
trainset_1: 3569 trainset_0:63684
trainset: 8566 validate: 9608
(8566, 1523)
epoch 0, vali: 0.835, vali_o: 0.863 zero: 0.833 *: 0.719 train: 0.855 test: 0.792 test_o: 0.648 zero: 0.792
epoch 1, vali: 0.879, vali_o: 0.912 zero: 0.877 *: 0.8 train: 0.902 test: 0.964 test_o: 0.574 zero: 0.965
